In [25]:
import pandas as pd
import numpy as np 

import speech_recognition as sr 

In [26]:
data = pd.read_json('data/train/manifest.jsonl', lines=True)

data

,id,audio_filepath,text,duration
0,73aff8bc8d99278c6ca6d1ac243557e3,files/73aff8bc8d99278c6ca6d1ac243557e3.wav,зили,2.298375
1,ed0f6706f75681a7915fec15d336aca5,files/ed0f6706f75681a7915fec15d336aca5.wav,дамира николаевича,3.800000
2,3c2875271fb918da312865549d444653,files/3c2875271fb918da312865549d444653.wav,антонине татариновой,2.500000
3,ba8443f3cc91e56667278db36dea02b7,files/ba8443f3cc91e56667278db36dea02b7.wav,виктору павловичу новокрещенову,3.040000
4,f3dba379c6280536aaa65a56c4358268,files/f3dba379c6280536aaa65a56c4358268.wav,жидкову,2.080000
...,...,...,...,...
79791,19fb11ac46e700873f80a0f615dad84c,files/19fb11ac46e700873f80a0f615dad84c.wav,вилка ложка,3.600000
79792,740fd944850faa097a05b2761ceb0665,files/740fd944850faa097a05b2761ceb0665.wav,фото таймырский,1.760000
79793,b14d3dd644d7f146433e1d478856a92d,files/b14d3dd644d7f146433e1d478856a92d.wav,айболит,1.700000
79794,ca27457fe19fe859f17a7c755d07b796,files/ca27457fe19fe859f17a7c755d07b796.wav,тольяттинский городской наркологический диспан...,3.440000


In [27]:
recog = sr.Recognizer()

i = np.random.randint(0, data.shape[0])
path = data['audio_filepath'].iloc[i]

with sr.AudioFile('data/train/'+path) as audio:
    rec = recog.record(audio)
    
result = recog.recognize_google(audio_data=rec, language='ru-RU') 
real = data['text'].iloc[i]

similarity = str(result).lower() == str(real).lower()

print(f'Recognized is "{str(result).lower()}",',  
      f'\nwhile real is "{str(real).lower()}".',  
      f'\nSimilarity is {similarity}')

Recognized is "бронь дастархан", 
while real is "бронь дастархан". 
Similarity is True


In [28]:
with open('models/vosk-model-small-ru-0.22/am/final.mdl') as model:
    print(model) 

<_io.TextIOWrapper name='models/vosk-model-small-ru-0.22/am/final.mdl' mode='r' encoding='cp1251'>


In [29]:
recog.recognize_sphinx(audio_data=rec, language='ru-RU')

RequestError: missing PocketSphinx language data directory: "C:\Users\honor\AppData\Local\Programs\Python\Python311\Lib\site-packages\speech_recognition\pocketsphinx-data\ru-RU"